1) Data preprocessing:
- description of the data
- visualization of the data: examples of audio signals and spectrograms, and examples of what seems to be meaw and bark patterns
- uniformization of the files regarding sampling rate, and number of channels
- (filtering of not instersting frequencies--- to reduce noise?) < ----- How to reduce noise? is it needed?

2) Feature Extraction:  creation of input data for the diferent classification procedures we'll explore:
- test and train dataframes with mel_spectograms + delta
- test and train dataframes with Mfccs + delta


In [20]:
import librosa
from matplotlib import pyplot as plt
import librosa.display
import IPython.display as ipd
import os
import audioread
from matplotlib.pyplot import specgram
import numpy as np
import pandas as pd
import re


#path for audio files folder:
relevant_path = './data/cats_dogs/'

# get files from directory and do all or a few, depending on range extracted below
sound_file_paths = [relevant_path+f for f in os.listdir(relevant_path)]
#print(sound_file_paths)

files_id=[]
pattern='(\w*_\d*).wav'
for n in sound_file_paths:
    files_id.append(re.search(pattern, n ).group(1))


#print(files_id)
n_cats=164 # number of files with cat sounds
n_dogs=113 # number of files with barking sounds    
catLabel=0
dogLabel=1

## Data Loading

In [21]:
# load data function:
# input: the path of the audio files
# Loads each audio file to a respective list depending of the file label.
# stores both number of channels and sampling rate into lists
# output: Cats and Dogs lists containing the loaded audio signals, number of channels and sampling rate lists for both cats and dogs

def load_data( files_path, sr=None): 
        
    cats = []
    cats_sr=[]
    channel_cats = []
    dogs = []
    channel_dogs = []
    dogs_sr=[]
   
# CREATE CODE TO GET CAT FILES AND DOG FILES AUTOMATICaly--> WITH RE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 
    n_cats=164 # number of files with cat sounds
    n_dogs=113 # number of files with barking sounds
    
    for i in range(n_cats):
       # print(files_path[i])
        cat_i, sr_i = librosa.load(files_path[i],sr=sr)
        cats.append(cat_i)
        cats_sr.append(sr_i)
        with audioread.audio_open(files_path[i]) as input_file:
            channel_cats.append(input_file.channels)
   
    for j in range(n_cats, n_cats + n_dogs):
       # print(files_path[j])
        dog_j, sr_j = librosa.load(files_path[j],sr=sr)
        dogs.append(dog_j)
        dogs_sr.append(sr_j)
        with audioread.audio_open(files_path[j]) as input_file:
            channel_dogs.append(input_file.channels)

    #channel_cats = list(channel_cats)
    #channel_dogs = list(channel_dogs)
    
    return cats, cats_sr, channel_cats, dogs, dogs_sr, channel_dogs


In [22]:
#load the files:

Cats_audio, cats_sr, channel_cats, Dogs_audio, dogs_sr, channel_dogs= load_data( sound_file_paths, sr=None)


## Data description

In [23]:
#Summary of data:

print('Number of Files per class:')
print('Cats audio files loaded: ' + str(len(Cats_audio)) + ' files')
print('Dogs audio files loaded: ' + str(len(Dogs_audio)) + ' files')
print('\n')


print('Number of Channels per class:')
cc=list(set(channel_cats))
cd=list(set(channel_dogs))
if len(cc)==1:
    print('all cat files have the same number of channels: ' + str(cc[0]) )
else: 
    print('cat files have different number of channels: ' + str(cc) )  
if len(cd)==1:
    print('all dog files have the same number of channels: ' + str(cd[0]) )
else: 
    print('dog files have different number of channels: ' + str(cd) )  
if cd!=cc:
    print("number of channels is different between cats and dogs")
else:
    n_channels=channel_cats[0]
print('\n')

print('Sampling rates per class:')
SRs_cat=list(set(cats_sr))
SRs_dog=list(set(dogs_sr))

if len(SRs_cat)==1:
    print('all cat files have the same sampling rate: ' + str(SRs_cat[0]) + ' samples per second' )
else: 
    print('cat files have different sampling rates: ' + str(SRs_cat)+' ----> should be normalized'  )  
if len(cd)==1:
    print('all dog files have the same sampling rate: ' + str(SRs_dog[0]) + ' samples per second'  )
else: 
    print('dog files have different sampling rate: ' + str(SRs_dog) + ' ----> should be normalized' )  

if SRs_cat!=SRs_dog:
    print("sample rates are different between cats and dogs files----> should be normalized")
else:
    sr=cats_sr[0]
    
print('\n')



Number of Files per class:
Cats audio files loaded: 164 files
Dogs audio files loaded: 113 files


Number of Channels per class:
all cat files have the same number of channels: 1
all dog files have the same number of channels: 1


Sampling rates per class:
all cat files have the same sampling rate: 16000 samples per second
all dog files have the same sampling rate: 16000 samples per second




In [24]:
import statistics  as st  

# duration of files:
length_cats=[]
length_dogs=[]

for c in Cats_audio:
    length_cats.append(librosa.core.get_duration(y=c, sr=sr))
for d in Dogs_audio:
    length_dogs.append(librosa.core.get_duration(y=d, sr=sr))


max_length_cats=max(length_cats)
min_length_cats=min(length_cats)
av_length_cats=st.mean(length_cats)
max_length_dogs=max(length_dogs)
min_length_dogs=min(length_dogs)
av_length_dogs=st.mean(length_dogs)
print('Duration of audio files per class:')
print('the average duration of cat files is: ' + str(av_length_cats) + ' seconds')
print('the min is: ' +str(min_length_cats) + ' and the max is '+ str(max_length_cats)+ 'seconds')
print('the average duration of Dog files is: ' + str(av_length_dogs) + ' seconds')
print('the min is: ' +str(min_length_dogs) + ' and the max is '+ str(max_length_dogs)+ 'seconds')


# what about amplitude of the audio signals?
# what about silence periods length and ocurrences?
# plot some distinct audios for cats and dogs:

Duration of audio files per class:
the average duration of cat files is: 8.072548399390245 seconds
the min is: 0.9233125 and the max is 17.976seconds
the average duration of Dog files is: 5.2959142699115045 seconds
the min is: 1.0156875 and the max is 17.2001875seconds


In [25]:
# Analysis in frequency!

# plot spectograms 
# isolate interesting patterns.


# extract mel_spectograms:
# Compute_mel_frequencies function:
# Computes the Mel-scaled spectrograms for each audio file
# input: lists of raw audio for cats and dogs, and sampling rate
#output: melspectograms for each file, in a list per class
def compute_mel_frequencies(cats,dogs, sr ):
    cats_mel_frequencies = []
    dogs_mel_frequencies = []
    
    for c in cats:
        cats_mel_frequencies.append(librosa.feature.melspectrogram(y=c,sr=sr))
    for d in dogs:
        dogs_mel_frequencies.append(librosa.feature.melspectrogram(y=d,sr=sr))
        
    return cats_mel_frequencies,dogs_mel_frequencies

def compute_MEL_deltas(cats_mel_frequencies,dogs_mel_frequencies):
    cats_mel_deltas = []
    dogs_dogs_deltas = []
    
    for i in range(164):
        cats_mel_deltas.append(librosa.feature.delta(cats_mel_frequencies[i]))
    for i in range(113):
        dogs_dogs_deltas.append(librosa.feature.delta(dogs_mel_frequencies[i]))
   
    return cats_mel_deltas, dogs_dogs_deltas

# extract MFCCs:
def compute_mfccs(cats_mel_frequencies,dogs_mel_frequencies,sr=22050):
    cats_mfccs = []
    dogs_mfccs = []
    
    for i in range(n_cats):
        cats_mfccs.append(librosa.feature.mfcc(S=librosa.power_to_db(cats_mel_frequencies[i]),sr=sr))
    for i in range(n_dogs):
        dogs_mfccs.append(librosa.feature.mfcc(S=librosa.power_to_db(dogs_mel_frequencies[i]),sr=sr))
        
    return cats_mfccs,dogs_mfccs

def compute_MFCC_deltas(cats_mfccs,dogs_mfccs):
    cats_deltas = []
    dogs_deltas = []
    
    for i in range(164):
        cats_deltas.append(librosa.feature.delta(cats_mfccs[i]))
    for i in range(113):
        dogs_deltas.append(librosa.feature.delta(dogs_mfccs[i]))
        
    return cats_deltas,dogs_deltas



In [26]:
cats_mel_frequencies,dogs_mel_frequencies=compute_mel_frequencies(Cats_audio,Dogs_audio , sr)
cats_mel_deltas, dogs_mel_deltas=compute_MEL_deltas(cats_mel_frequencies,dogs_mel_frequencies)
cats_mfccs, dogs_mfccs= compute_mfccs(cats_mel_frequencies,dogs_mel_frequencies,sr)
cats_mfcc_delta, dogs_mfcc_delta=compute_MFCC_deltas(cats_mfccs,dogs_mfccs)

In [27]:

#to access a specific cell > dataframe.iloc[row]['column?name']


# GENERIC DATAFRAME that accepts any size features:
#features_name='mel', 'mel_delta', 'mfccs', 'mfcc_delta'
#features cats is a list with the features extracted for each file.
def build_features_Dataframe(features_name, features_cats, features_dogs, files_id, removeSilentFrames=False, labelSilentFrames=False):
        
    df_cats=pd.DataFrame()
    for f in range(0, len(features_cats)):
        df_filec=pd.DataFrame([[np.transpose(features_cats[f])[0]]])
        for frame_mels in range(1,np.transpose(features_cats[f]).shape[0]):
            df_filec=df_filec.append([[np.transpose(features_cats[f])[frame_mels]]], ignore_index= True)
   
        df_filec['file_id']=files_id[f]
        df_cats=df_cats.append(df_filec)    

    df_cats['label']=catLabel
    df_cats.columns = [features_name, 'File_id', 'Label']

    df_dogs=pd.DataFrame()
    for fd in range(0, len(features_dogs)):
        df_file=pd.DataFrame([[np.transpose(features_dogs[fd])[0]]])
        for frame_mels in range(1,np.transpose(features_dogs[fd]).shape[0]):
            df_file=df_file.append([[np.transpose(features_dogs[fd])[frame_mels]]], ignore_index= True)

        df_file['file_id']=files_id[fd+n_cats]
        df_dogs=df_dogs.append(df_file)    

    df_dogs['label']=dogLabel
    df_dogs.columns = [features_name, 'File_id', 'Label']
    return df_cats, df_dogs



In [28]:
df_cats_melDeltas, df_dogs_melDeltas =build_features_Dataframe('Mel_deltas',cats_mel_deltas, dogs_mel_deltas, files_id )
df_cats_mel, df_dogs_mel =build_features_Dataframe('Mel',cats_mel_frequencies, dogs_mel_frequencies, files_id )
df_cats_MFCCDeltas, df_dogs_MFCCDeltas =build_features_Dataframe('MFCC_deltas',cats_mfcc_delta, dogs_mfcc_delta, files_id )
df_cats_MFCC, df_dogs_MFCC =build_features_Dataframe('MFCC',cats_mfccs, dogs_mfccs, files_id )

df_cats_melANDdeltas = pd.merge(df_cats_mel, df_cats_melDeltas )
df_cats_MFCC_AND_Deltas = pd.merge(df_cats_MFCC, df_cats_MFCCDeltas )
df_dogs_melANDdeltas = pd.merge(df_dogs_mel, df_dogs_melDeltas )
df_dogs_MFCC_AND_Deltas = pd.merge(df_dogs_MFCC, df_dogs_MFCCDeltas )

In [29]:
n_dog_frames=df_dogs_melDeltas.shape[0]
n_cat_frames=df_cats_melDeltas.shape[0]
print('Number of frames per class:')
print('the audio files of cats make in total: '+ str(n_cat_frames)+' frames' )
print('the audio files of dogs make in total: '+ str(n_dog_frames)+' frames')


#we have to take this aspect in consideration when splitiing the data into test and train: we want a ballanced training set!


Number of frames per class:
the audio files of cats make in total: 41475 frames
the audio files of dogs make in total: 18761 frames


In [30]:
import random as rndm
from random import shuffle

def split_dataframes(cats_dataframe,dogs_dataframe,test_size=0.3):

    #keep unbalanced dataset in the training for now: 0.7*164 cats+ 0.7*113 dogs
    n_train_files_cats=(int((1-test_size)*n_cats))
    n_train_files_dogs=(int((1-test_size)*n_dogs))
    
    # select 30% as test and 70% as train at file level! 
    files_id_cats=files_id[0:n_cats]
    rndm.shuffle(files_id_cats)

    files_id_cats_train=[]
    files_id_cats_test=[]
    for i in range(0,n_train_files_cats):
        files_id_cats_train.append(files_id_cats[i])
    files_id_cats_test=files_id_cats[n_train_files_cats:-1]

    files_id_dogs=files_id[n_cats:n_cats+n_dogs]
    rndm.shuffle(files_id_dogs)    
    files_id_dogs_train=[]
    files_id_dogs_test=[]
    for i in range(0,n_train_files_dogs):
        files_id_dogs_train.append(files_id_dogs[i])
    files_id_dogs_test=files_id_dogs[n_train_files_dogs:-1]

    dftrain_cats=cats_dataframe.loc[cats_dataframe['File_id'].isin(files_id_cats_train)]
    dftest_cats=cats_dataframe.loc[cats_dataframe['File_id'].isin(files_id_cats_test)]
    dftrain_dogs=dogs_dataframe.loc[dogs_dataframe['File_id'].isin(files_id_dogs_train)]
    dftest_dogs=dogs_dataframe.loc[dogs_dataframe['File_id'].isin(files_id_dogs_test)]

    # concatenate in two dataframe test and train.
    df_TRAIN=pd.DataFrame()
    df_TEST=pd.DataFrame()
    df_TRAIN=df_TRAIN.append(dftrain_cats)
    df_TRAIN=df_TRAIN.append(dftrain_dogs)
    df_TEST=df_TEST.append(dftest_cats)
    df_TEST=df_TEST.append(dftest_dogs)


    # # shuffle...
    df_TEST=df_TEST.sample(frac=1)
    df_TRAIN=df_TRAIN.sample(frac=1)
    return df_TRAIN, df_TEST

In [31]:
df_TRAIN, df_TEST = split_dataframes( df_cats_melANDdeltas, df_dogs_melANDdeltas, test_size=0.3)

df_TEST

,Mel,File_id,Label,Mel_deltas
9722400,"[0.00016582285722774185, 0.001549604064480714,...",cat_57,0,"[-0.00015438764311580705, 0.000279660455320938..."
13265772,"[29.161712062997076, 1.7148953685096622, 0.832...",cat_90,0,"[4.199752653112108, -0.5210485970736605, 0.085..."
12444953,"[0.12526863185471349, 0.02057436558645539, 0.0...",cat_81,0,"[-0.0983062486751754, 0.009155016011201979, 0...."
17143,"[0.08771724466696658, 0.7825560998900158, 0.96...",cat_1,0,"[-6.99431033121676e-05, -0.0034429924538508616..."
336204,"[0.00035328334984479254, 0.0001495868529268162...",dog_barking_1,1,"[2.533343473114303e-07, 2.381810507147274e-06,..."
319415,"[0.000479898877169932, 0.000393819685700221, 0...",dog_barking_1,1,"[-3.613428949904808e-08, -1.8919087429593882e-..."
387568,"[0.0002915960702539891, 0.0001722061259673978,...",dog_barking_1,1,"[-2.5921720520515252e-05, -9.260029308081066e-..."
4458749,"[14.416175424431964, 3.5616713169701844, 1.018...",cat_159,0,"[0.11587110728531297, 0.0112940708892586, -0.0..."
10467806,"[0.22162237223817335, 0.15688999730567604, 0.0...",cat_61,0,"[0.008162756078408806, 0.0022144688554962496, ..."
5374754,"[0.018367086395735414, 0.04832104529484257, 0....",cat_165,0,"[-0.004860355153905314, -0.02144134330246274, ..."


In [32]:
df_TRAIN.to_pickle('.\Features_sets/'+ 'Train_MELandDeltas.pkl')  
df_TEST.to_pickle('.\Features_sets/'+ 'Test_MELandDeltas.pkl')


In [36]:
#def load_features(train_set_path, test_set_path, window_size, overlap):
train_set_path='.\Features_sets/'+ 'Train_MELandDeltas.pkl'
test_set_path='.\Features_sets/'+ 'Test_MELandDeltas.pkl'
df_train=pd.read_pickle(train_set_path)
df_test=pd.read_pickle(test_set_path)


# filelist = glob.glob('features_mel_spectrograms/*.npy')
# labels = []
# data = []

# for file in filelist:
#     nfile = np.load(file)
#     if 'cat' in file:
#         label = 0 #'cat'
#     else:
#         label = 1 #'dog'
#     crop = int(nfile.shape[1] / 28)
#     for i in range(crop):
#         labels.append(label)
#         data.append(nfile[:,i*28:(i+1)*28])

# X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.1, random_state=59)
# # return X_train, X_test, y_train, y_test
# #return np.array(X_train, dtype=np.float32), np.array(X_test, dtype=np.float32), np.array(y_train), np.array(y_test)



# def load_features():
#     filelist = glob.glob('features_mel_spectrograms/*.npy')
#     labels = []
#     data = []

#     for file in filelist:
#         nfile = np.load(file)
#         if 'cat' in file:
#             label = 0 #'cat'
#         else:
#             label = 1 #'dog'
#         crop = int(nfile.shape[1] / 28)
#         for i in range(crop):
#             labels.append(label)
#             data.append(nfile[:,i*28:(i+1)*28])

#     X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.1, random_state=59)
#     # return X_train, X_test, y_train, y_test
#     return np.array(X_train, dtype=np.float32), np.array(X_test, dtype=np.float32), np.array(y_train), np.array(y_test)


In [37]:
df_test.keys()
mels_train= df_test.values[0][0]

In [ ]:
print(114+49)
print(files_id_cats_train[-1])
print(files_id_cats_test[0])

In [ ]:
# # create features Dataframe with melfrequencies:
# # rows are frames

# catLabel=0
# dogLabel=1

# df_cats=pd.DataFrame()
# for f in range(0, len(cats_mel_frequencies)):
#     df_filec=pd.DataFrame([[np.transpose(cats_mel_frequencies[f])[0]]])
#     for frame_mels in range(1,np.transpose(cats_mel_frequencies[f]).shape[0]):
#         df_filec=df_filec.append([[np.transpose(cats_mel_frequencies[f])[frame_mels]]], ignore_index= True)
   
#     df_filec['file_id']=files_id[f]
#     df_cats=df_cats.append(df_filec)    
    
# df_cats['label']=catLabel
# df_cats.columns = ['Features_128melfrequencies', 'File_id', 'Label']

# df_dogs=pd.DataFrame()
# for fd in range(0, len(dogs_mel_frequencies)):
#     df_file=pd.DataFrame([[np.transpose(dogs_mel_frequencies[fd])[0]]])
#     for frame_mels in range(1,np.transpose(dogs_mel_frequencies[fd]).shape[0]):
#         df_file=df_file.append([[np.transpose(dogs_mel_frequencies[fd])[frame_mels]]], ignore_index= True)
   
#     df_file['file_id']=files_id[fd+n_cats]
#     df_dogs=df_dogs.append(df_file)    
    
# df_dogs['label']=dogLabel
# df_dogs.columns = ['Features_128melfrequencies', 'File_id', 'Label']

In [ ]:
# # create features Dataframe with melfrequencies:
# # rows are frames

# catLabel=0
# dogLabel=1
# # mel_frequencies_dataframes Function:
# # creates two dataframes containing the melfrequencies, label and file id of each frame.
# # input: computed mel frequencies , identification of files, possibility of removing silent frames



#                                     #TODO 
# # remove or label differently the silent frames?
# # generalize function to accept other features, as deltas or mfccs, or mfccs + deltas

# def mel_frequencies_dataframes(cats_mel_frequencies, dogs_mel_frequencies,files_id, removeSilentFrames=False ):
#     df_cats=pd.DataFrame()
#     for f in range(0, len(cats_mel_frequencies)):
#         df_filec=pd.DataFrame([[np.transpose(cats_mel_frequencies[f])[0]]])
#         for frame_mels in range(1,np.transpose(cats_mel_frequencies[f]).shape[0]):
#             df_filec=df_filec.append([[np.transpose(cats_mel_frequencies[f])[frame_mels]]], ignore_index= True)
   
#         df_filec['file_id']=files_id[f]
#         df_cats=df_cats.append(df_filec)    

#     df_cats['label']=catLabel
#     df_cats.columns = ['Features_128melfrequencies', 'File_id', 'Label']

#     df_dogs=pd.DataFrame()
#     for fd in range(0, len(dogs_mel_frequencies)):
#         df_file=pd.DataFrame([[np.transpose(dogs_mel_frequencies[fd])[0]]])
#         for frame_mels in range(1,np.transpose(dogs_mel_frequencies[fd]).shape[0]):
#             df_file=df_file.append([[np.transpose(dogs_mel_frequencies[fd])[frame_mels]]], ignore_index= True)

#         df_file['file_id']=files_id[fd+n_cats]
#         df_dogs=df_dogs.append(df_file)    

#     df_dogs['label']=dogLabel
#     df_dogs.columns = ['Features_128melfrequencies', 'File_id', 'Label']
#     return df_cats, df_dogs